## Household_Summary
 
VIF 확인해보기

In [1]:
# Library
import warnings
warnings.filterwarnings("ignore")
import os
import random
import pickle
import gc
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.cluster import KMeans, MiniBatchKMeans
# from category_encoders import TargetEncoder
from sklearn.preprocessing import (
    StandardScaler, PowerTransformer, OrdinalEncoder,
    OneHotEncoder, FunctionTransformer, PolynomialFeatures, LabelEncoder,
)
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import (
    LogisticRegression, LinearRegression, Ridge, Lasso,
    SGDRegressor, ElasticNet
)
from sklearn.model_selection import (
    train_test_split, cross_val_score, cross_validate,
    GridSearchCV, KFold, cross_val_predict
)
from sklearn.metrics import (
    roc_auc_score, mean_squared_error, make_scorer, accuracy_score, log_loss
)
from sklearn import set_config, datasets
# from catboost import (
#     CatBoostRegressor, CatBoostClassifier,
# )
# import category_encoders as ce
# from sklearn.pipeline import (
#     Pipeline, FeatureUnion, make_pipeline
# )
from sklearn.ensemble import (
    RandomForestClassifier, StackingClassifier, StackingRegressor,
    GradientBoostingRegressor, VotingClassifier, VotingRegressor,
    HistGradientBoostingRegressor, GradientBoostingClassifier,
    BaggingClassifier, AdaBoostClassifier, RandomForestRegressor,ExtraTreesRegressor
)
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.svm import SVC, SVR, LinearSVC
from xgboost import XGBRegressor, XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons

import xgboost as xgb
import lightgbm as lgb
import re
import math
# import optuna
from pandas.errors import DataError
from scipy.stats import zscore

%matplotlib inline

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything(42)

In [3]:
# 데이터 로드
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# train = pd.read_csv("/content/drive/MyDrive/3-2/머신러닝/dacon/소득 예측 AI/train.csv")
# test = pd.read_csv("/content/drive/MyDrive/3-2/머신러닝/dacon/소득 예측 AI/test.csv")

In [4]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# vif = pd.DataFrame()
# vif['vif factor'] = [variance_inflation_factor(trainx.values,i) for i in range(trainx.shape[1])]
# vif['features'] = trainx.columns
# vif= vif.sort_values('vif factor').reset_index(drop=True)
# vif

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
# import sweetviz as sv
# # Sweetviz 보고서 생성

# report_train = sv.analyze(train)
# report_train.show_html("report_train.html")

# report_test = sv.analyze(test)
# report_test.show_html("reprot_test.html")

### Associations  

## Numerical
Working_Week (Yearly)  
Age  
Losses

## Categorical
Industry_Status  
Occupation_Status  


## vif   
Industry_Status - Working_Week  
Occupation_Status - Working_Week  
Martial_Status - Age  
Household_Status - Age  
Household_Summary - Ahe  
Birth_country - Citizenship  
Tax_Status - Working_Week

In [7]:
#전처리
def pre_all(train, test):
    print(f"전처리 전 train 크기 : {train.shape}")
    print(f"전처리 전 test 크기 : {test.shape}")
    print("=================전처리 중=================")

    df = pd.concat([train,test]).reset_index(drop = True)

    df['Working_Week (Yearly)_Age1'] = df['Working_Week (Yearly)'] * df['Age']
    df['Working_Week (Yearly)_Age2'] = df['Working_Week (Yearly)'] + df['Age']

    df['Working_Week (Yearly)_Losses1'] = df['Working_Week (Yearly)'] * df['Losses']
    df['Working_Week (Yearly)_Losses2'] = df['Working_Week (Yearly)'] + df['Losses']  

    df['Working_Week (Yearly)_Gains1'] = df['Working_Week (Yearly)'] * df['Gains']
    df['Working_Week (Yearly)_Gains2'] = df['Working_Week (Yearly)'] + df['Gains']

    df['Age_Losses1'] = df['Age'] * df['Losses']
    df['Age_Losses2'] = df['Age'] + df['Losses']

    df['Age_Gains1'] = df['Age'] * df['Gains']
    df['Age_Gains2'] = df['Age'] + df['Gains']

    df['Household_Status'].fillna('NAN', inplace=True)
    
    df['Industry_Occupation'] = df['Industry_Status'] + df['Occupation_Status']
    df['Industry_Tax'] = df['Industry_Status'] + df['Tax_Status']
    df['Industry_Education'] = df['Industry_Status'] + df['Education_Status']
    df['Industry_Household'] = df['Industry_Status'] + df['Household_Status']

    df['Occupation_Gender'] = df['Occupation_Status'] + df['Gender'] 
    # df['Industry_Gender'] = df['Industry_Status'] + df['Gender'] 
    # df['Occupation_Income'] = df['Occupation_Status'] + df['Income_Status'] 
    df['Gender_Household'] = df['Gender'] + df['Household_Status'] 

    train = df[~df["Income"].isnull()].sort_values("ID").reset_index(drop = True)
    test = df[df["Income"].isnull()].sort_values("ID").reset_index(drop=True)
    test = test.drop(columns=["Income"])

    print(f"전처리 후 train 크기 : {train.shape}")
    print(f"전처리 후 test 크기 : {test.shape}")

    return train, test

train_pre, test_pre = pre_all(train, test)

전처리 전 train 크기 : (20000, 23)
전처리 전 test 크기 : (10000, 22)
=================전처리 중=================
전처리 후 train 크기 : (20000, 39)
전처리 후 test 크기 : (10000, 38)


In [8]:
from catboost import CatBoostRegressor, Pool
categorical_features = []
for i in train_pre.columns:
    if train_pre[i].dtype == object:
        categorical_features.append(i)
categorical_features.remove('ID')

for i in categorical_features:
    train_pre[i] = train_pre[i].astype('category')
    test_pre[i] = test_pre[i].astype('category')

In [9]:
x_train = train_pre.drop(['Income', 'ID'], axis=1)
target = train_pre['Income']
test_ID = test['ID']

In [10]:
pool_train = Pool(data=x_train, label=target, cat_features=categorical_features)

In [11]:
import shap 

X_importance = test_pre.drop(columns=['ID'])

model = CatBoostRegressor(random_state=42).fit(pool_train)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_importance)
shap_sum = np.abs(shap_values).mean(axis=0)


Learning rate set to 0.065727
0:	learn: 690.7977305	total: 131ms	remaining: 2m 11s
1:	learn: 681.1316214	total: 160ms	remaining: 1m 19s
2:	learn: 672.4198598	total: 193ms	remaining: 1m 4s
3:	learn: 664.6461543	total: 240ms	remaining: 59.7s
4:	learn: 657.5859526	total: 275ms	remaining: 54.8s
5:	learn: 651.3075121	total: 305ms	remaining: 50.6s
6:	learn: 645.9312449	total: 340ms	remaining: 48.2s
7:	learn: 640.8849151	total: 369ms	remaining: 45.8s
8:	learn: 636.3888742	total: 398ms	remaining: 43.8s
9:	learn: 632.3584347	total: 429ms	remaining: 42.5s
10:	learn: 628.9767738	total: 460ms	remaining: 41.4s
11:	learn: 625.8650404	total: 499ms	remaining: 41.1s
12:	learn: 622.9925732	total: 534ms	remaining: 40.5s
13:	learn: 620.2908685	total: 572ms	remaining: 40.3s
14:	learn: 617.7693762	total: 606ms	remaining: 39.8s
15:	learn: 615.9930892	total: 638ms	remaining: 39.3s
16:	learn: 613.6729463	total: 673ms	remaining: 38.9s
17:	learn: 611.8618644	total: 708ms	remaining: 38.6s
18:	learn: 610.3551268	t

In [12]:
shap_val = shap_sum.tolist()
sum(shap_val)

624.9063982795864

In [13]:
importance_df = pd.DataFrame([X_importance.columns.tolist(), shap_sum.tolist()]).T
importance_df.columns = ['column_name', 'shap_importance']
importance_df = importance_df.sort_values('shap_importance', ascending=False)
importance_df 

,column_name,shap_importance
5,Industry_Status,75.777336
6,Occupation_Status,75.291189
35,Occupation_Gender,55.569429
21,Working_Week (Yearly)_Age1,45.754017
31,Industry_Occupation,40.284791
3,Employment_Status,32.936934
1,Gender,27.527181
2,Education_Status,26.75951
33,Industry_Education,26.597456
16,Tax_Status,19.808502


In [14]:
# # 지정된(SHAP_THRESHOLD) Shap feature 중요도 이상인 것만 선택
# SHAP_THRESHOLD = 0.01 
# features_selected = importance_df.query('shap_importance > @SHAP_THRESHOLD').column_name.tolist()
# x_train = x_train[features_selected]
# x_test = test_pre[features_selected]

# print(x_train.shape)
# print(x_test.shape)

In [15]:
clf = CatBoostRegressor(random_state = 42)
clf.fit(pool_train)

Learning rate set to 0.065727
0:	learn: 690.7977305	total: 49.8ms	remaining: 49.7s
1:	learn: 681.1316214	total: 82.9ms	remaining: 41.4s
2:	learn: 672.4198598	total: 114ms	remaining: 37.9s
3:	learn: 664.6461543	total: 145ms	remaining: 36.1s
4:	learn: 657.5859526	total: 180ms	remaining: 35.8s
5:	learn: 651.3075121	total: 212ms	remaining: 35.1s
6:	learn: 645.9312449	total: 246ms	remaining: 34.9s
7:	learn: 640.8849151	total: 278ms	remaining: 34.4s
8:	learn: 636.3888742	total: 305ms	remaining: 33.6s
9:	learn: 632.3584347	total: 335ms	remaining: 33.2s
10:	learn: 628.9767738	total: 365ms	remaining: 32.9s
11:	learn: 625.8650404	total: 399ms	remaining: 32.8s
12:	learn: 622.9925732	total: 432ms	remaining: 32.8s
13:	learn: 620.2908685	total: 465ms	remaining: 32.8s
14:	learn: 617.7693762	total: 499ms	remaining: 32.7s
15:	learn: 615.9930892	total: 530ms	remaining: 32.6s
16:	learn: 613.6729463	total: 562ms	remaining: 32.5s
17:	learn: 611.8618644	total: 595ms	remaining: 32.5s
18:	learn: 610.3551268	t

In [16]:
test_pool = Pool(data=test_pre.drop(columns = ['ID']), cat_features=categorical_features)
pred = clf.predict(test_pool)
pred = [0 if i < 0 else i for i in pred]

In [17]:
t = pd.Timestamp.now()
fname = f"Income_submission_{t.month:02}월{t.day:02}일{t.hour:02}시{t.minute:02}분.csv"
pd.DataFrame({'ID': test_ID, 'Income': pred}).to_csv(fname, index=False)
print(f"'{fname}' is ready to submit.")

'Income_submission_03월24일15시07분.csv' is ready to submit.
